In [1]:
%%time

import json
reviews = []
with open("../data/review.json") as f:
    index = 0
    for line in f:
        if index % 1000000 == 0:
            print(index)
        index += 1
        reviews.append(json.loads(line))

0
1000000
2000000
3000000
4000000
5000000
6000000
CPU times: user 46.2 s, sys: 6.47 s, total: 52.7 s
Wall time: 53.6 s


In [2]:
print(reviews[0])

{'review_id': 'Q1sbwvVQXV2734tPgoKj4Q', 'user_id': 'hG7b0MtEbXx5QzbzE6C_VA', 'business_id': 'ujmEBvifdJM6h6RLv4wQIg', 'stars': 1.0, 'useful': 6, 'funny': 1, 'cool': 0, 'text': 'Total bill for this horrible service? Over $8Gs. These crooks actually had the nerve to charge us $69 for 3 pills. I checked online the pills can be had for 19 cents EACH! Avoid Hospital ERs at all costs.', 'date': '2013-05-07 04:34:36'}


In [3]:
from collections import Counter
prolific_reviewers = Counter([review['user_id'] for review in reviews]).most_common(50)

In [4]:
keep_ids = {pr[0] : 0 for pr in prolific_reviewers}

In [5]:
by_author = {} # author : "review 1\n review 2\n review 3"
for review in reviews:
    uid = review['user_id']
    if uid in keep_ids:
        uid = review['user_id']
        if uid in by_author:
            by_author[uid] += "\n{}".format(review['text'])
        else:
            by_author[uid] = "{}".format(review['text'])

In [6]:
len(by_author)

50

In [7]:
# check that we have at least 200000 characters for each author
sorted([(len(by_author[key]), key) for key in by_author])[:5]

[(218976, 'ic-tyi1jElL_umxZVh8KNA'),
 (237086, 'PcvbBOCOcs6_suRDH7TSTg'),
 (288579, 'PeLGa5vUR8_mcsn-fn42Jg'),
 (467816, 'Lfv4hefW1VbvaC2gatTFWA'),
 (468683, 'iDlkZO2iILS8Jwfdy7DP9A')]

In [8]:
def get_chunks(l, n):
    n = max(1, n)
    return [l[i:i+n] for i in range(0, len(l), n)]

In [9]:
train_texts = []  # the first 100 000 chars from each author
train_labels = [] # each author
test_texts = []   # 100 texts of 1000 characters each (second 100 000 chars of each author)
test_labels = []  # each author * 100

author_int = {author: i for i,author in enumerate(by_author)}
int_author = {author_int[author]: author for author in author_int}

for author in by_author:
    train_text = by_author[author][:50000]
    train_label = author_int[author]
    train_texts.append(train_text)
    train_labels.append(train_label)
    
    short_texts = get_chunks(by_author[author][50000:100000], 1000)
    for text in short_texts:
        test_texts.append(text)
        test_labels.append(author_int[author])

In [12]:
train_data = {
    "train_texts": train_texts,
    "train_labels": train_labels
}
test_data = {
    "test_texts": test_texts,
    "test_labels": test_labels
}

In [13]:
import pickle

with open("../data/train_data.pickle", "wb") as f:
    pickle.dump(train_data, f)
with open("../data/test_data.pickle", "wb") as f:
    pickle.dump(test_data, f)